# Imports

In [1]:
import pandas as pd
import numpy as np

---

In [16]:
# Columnas a cargar
columnas_cargar = ["date", "event_id","ref_type","ref_hash","application_id","attributed", "session_user_agent", "kind", "wifi"]
# Definicion de tipos
tipos = {"event_id" : "int16",
         "ref_hash" : "int64",
         "application_id" : "int16",
         "attributed": "bool",
         "wifi":"bool"}

events = pd.read_csv("../../../../data/tp2/events.csv", usecols=columnas_cargar, dtype=tipos, parse_dates=["date"])


### Formateo el sistema operativo

In [17]:
events['os'] = 1 * ((events["ref_type"] == 1891515180541284343).astype('int8'))

In [13]:
events.drop(columns = {"ref_type"},inplace = True)

In [18]:
events["os"].value_counts()

1    6421584
0    1322997
Name: os, dtype: int64

### 1 ---> ANDROID 
### 0 ---> IOS

In [42]:
events.sort_values(by = "date",inplace = True)

In [62]:
events["dayofweek"] = (events["date"].dt.dayofweek).astype("int8")

In [26]:
def armar_ventanas(dataset,col_fecha):
    """Recibe el dataset y una string indicando el 
    nombre de la columna que lleva la fecha"""
    
    ventanas = []
    for x in range(5):
        ventanas.append((dataset.loc[((18+x) <= dataset[col_fecha].dt.day) & (dataset[col_fecha].dt.day <= (20+x))]))
    return ventanas

In [70]:
def target_ventanas(ventanas):
    entrenamientos = []
    for ventana in ventanas:
        ent = ventana["ref_hash"].drop_duplicates().to_frame()
        ent.rename(columns ={"ref_hash":"device_id"},inplace = True)
        ent.set_index("device_id", inplace=True)
        entrenamientos.append(ent)
    return entrenamientos

In [63]:
ventanas = armar_ventanas(events,"date")

In [71]:
targets = target_ventanas(ventanas)

## Comienzo a trabajar con la primer ventana

In [64]:
ventana1 = ventanas[0]

In [105]:
entrenamiento1 = targets[0]

In [86]:
ventana1.head()

,date,event_id,ref_type,ref_hash,application_id,attributed,session_user_agent,kind,wifi,os,dayofweek
170092,2019-04-18 00:00:00.027,13,1891515180541284343,4314242242878368557,65,False,3.819516e+18,4.853173e+18,True,1,3
947821,2019-04-18 00:00:00.071,158,1494519392962156891,2068670382964280534,230,False,3.819516e+18,8.370513e+18,True,0,3
1724915,2019-04-18 00:00:00.102,15,1891515180541284343,726218951290512705,65,False,3.819516e+18,5.500848e+18,True,1,3
2794789,2019-04-18 00:00:00.309,0,1891515180541284343,1329210878469536477,263,False,3.819516e+18,5.882882e+18,False,1,3
5526010,2019-04-18 00:00:00.486,2,1891515180541284343,5661732594213028190,210,False,3.819516e+18,6.168309e+18,False,1,3


## Promedio de eventos por día

In [100]:
feature = ventana1.groupby(["ref_hash","dayofweek"]).size()\
                  .groupby("ref_hash").mean()\
                  .to_frame().rename(columns = {0:"promedio_eventos"})

In [104]:
feature["promedio_eventos"] = feature["promedio_eventos"].astype("int64")

In [108]:
entrenamiento1 = entrenamiento1.merge(feature, how = 'left',left_index = True,right_index = True)

## Evento más frecuente

In [149]:
feature = ventana1.groupby(["ref_hash","event_id"]).size().to_frame()
                  

In [158]:
idx = feature.groupby(['ref_hash'])[0].transform(max) == feature[0]
feature = feature[idx]

In [211]:
feature.rename(columns= {0:"cant_ocurrencias"},inplace = True)

In [212]:
feature.head()

,,cant_ocurrencias
ref_hash,event_id,
41863526108385,1,52
161514654074162,23,4
186034136943920,1,10
360710529886978,23,2
365882020742330,15,41


### ¿Hay dispositivos que tengan más de un evento como máximo ?

In [201]:
((feature.groupby(["ref_hash"]).count()[0]) > 1).sum()

27801

### ¿Cual es el máximo numero de eventos empatados como máximo por dispositivo?

In [196]:
(feature.groupby(["ref_hash"]).count()[0]).max()

10

### ¿Qué criterio se toma para decidir con cual evento quedarse? 

#### Voy a crear una lista ordenada de los eventos más presentes por dispositivo, contando un solo dispositivo por evento para no desbalancear si un dispositivo hace 10000 el mismo evento. Esto podría dar un orden de importancia entre eventos

In [203]:
top_eventos = ventana1.groupby(["ref_hash","event_id"]).size().reset_index()\
                      .groupby("event_id").size().sort_values(ascending = False)

In [204]:
top_eventos.head()

event_id
23    63062
15    45078
7     32720
0     25747
2     20475
dtype: int64

In [202]:
top_eventos = top_eventos.index.to_list()